Import Libraries

In [1]:
from typing import List
import match_module
import numpy as np

In [2]:
def get_images_paths(txt_filename: str) -> List[str]:
    with open(txt_filename) as fp:
        images_paths = fp.readlines()

    images_paths = [x.strip() for x in images_paths]

    return images_paths

In [3]:
def get_model_images_paths() -> List[str]:
    return get_images_paths('model.txt')


In [4]:
def get_query_images_paths():
    return get_images_paths('query.txt')


In [5]:
def compute_recognition_rate(dist_type, hist_type, num_bins, model_images_paths, query_images_paths):
    best_matches, _ = match_module.find_best_match(model_images_paths, query_images_paths, dist_type, hist_type,
                                                   num_bins)
    num_query_images = len(query_images_paths)
    num_correct_matches = sum(best_matches == range(num_query_images))
    recognition_rate = num_correct_matches / num_query_images
    print("\n\tRecognition rate for: dist_type=%s, hist_types=%s, num_bins=%d"
          "\n\t%d" % (dist_type, hist_type, num_bins, recognition_rate))

    return recognition_rate, num_correct_matches, (dist_type, hist_type, num_bins)

In [9]:
dists_types: List[str] = ['intersect','chi2', 'l2'] 
hists_types: List[str] = ['grayvalue', 'rgb', 'rg', 'dxdy']
list_num_bins: List[int] = [5, 10, 20, 25, 30, 35, 40, 45, 50]

num_correct_matches: int = 0
best_recognition_rate: float = -1.0
best_combination = ['Nan', 'Nan', -1]

model_images_paths: List[str] = get_model_images_paths()
query_images_paths: List[str] = get_query_images_paths()

In [10]:
from concurrent import futures
ex = futures.ThreadPoolExecutor(max_workers=10)

wait_for = [
    ex.submit(compute_recognition_rate, dist_type, hist_type, num_bins, model_images_paths, query_images_paths)
    for dist_type in dists_types for hist_type in hists_types for num_bins in list_num_bins
]

results = []

for f in futures.as_completed(wait_for):
    result = f.result()
    print('main: result: {}'.format(result))
    results.append(result)


	Recognition rate for: dist_type=intersect, hist_types=rgb, num_bins=5
	0
main: result: (0.7752808988764045, 69, ('intersect', 'rgb', 5))

	Recognition rate for: dist_type=intersect, hist_types=grayvalue, num_bins=45
	0
main: result: (0.5280898876404494, 47, ('intersect', 'grayvalue', 45))

	Recognition rate for: dist_type=intersect, hist_types=grayvalue, num_bins=40
	0
main: result: (0.5280898876404494, 47, ('intersect', 'grayvalue', 40))

	Recognition rate for: dist_type=intersect, hist_types=grayvalue, num_bins=20
	0
main: result: (0.5168539325842697, 46, ('intersect', 'grayvalue', 20))

	Recognition rate for: dist_type=intersect, hist_types=grayvalue, num_bins=35
	0
main: result: (0.5168539325842697, 46, ('intersect', 'grayvalue', 35))

	Recognition rate for: dist_type=intersect, hist_types=grayvalue, num_bins=5
	0
main: result: (0.4157303370786517, 37, ('intersect', 'grayvalue', 5))
	Recognition rate for: dist_type=intersect, hist_types=grayvalue, num_bins=25
	0

main: result: (0

In [11]:
sorted_results = np.sort(results, axis=0)
